In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2023-ml-w6p1/submit.csv
/kaggle/input/2023-ml-w6p1/train.csv
/kaggle/input/2023-ml-w6p1/test.csv
/kaggle/input/2023-ml-w6p1/train_label.csv


In [2]:
import random
random.seed(123)

In [3]:
x = pd.read_csv("/kaggle/input/2023-ml-w6p1/train.csv")
test = pd.read_csv("/kaggle/input/2023-ml-w6p1/test.csv")
y = pd.read_csv("/kaggle/input/2023-ml-w6p1/train_label.csv")
submit = pd.read_csv("/kaggle/input/2023-ml-w6p1/submit.csv")

In [4]:
y.drop(['id'],axis=1,inplace=True)

In [5]:
y = np.squeeze(y)

In [6]:
group_num = {}
for group in np.unique(y):
    num = np.sum(y == group)
    group_num[group] = num

In [7]:
min(group_num.values()) #독립변수가 26보다 작아야함.

26

In [8]:
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score

In [9]:
def get_random_columns(df, n):
    """
    주어진 데이터프레임에서 랜덤으로 n개의 칼럼을 선택하여 반환하는 함수입니다.
    
    Parameters:
        - df: 데이터프레임
        - n: 선택할 칼럼의 수
        
    Returns:
        - 선택된 칼럼들로 이루어진 새로운 데이터프레임
    """
    cols = df.columns.tolist()
    random.shuffle(cols)  # 칼럼들을 랜덤하게 섞음
    return df[cols[:n]]  # 상위 n개의 칼럼만 선택하여 반환

In [10]:
def get_LDA(x,y):
    X_train,X_test,Y_train,Y_test=train_test_split(x,y, test_size=0.2, random_state=42, stratify=y)
    
    ss = StandardScaler()
    X_train_ss = ss.fit_transform(X_train)
    X_test_ss = ss.transform(X_test)
    
    lda_ss = LinearDiscriminantAnalysis()
    lda_ss.fit(X_train_ss, Y_train) # LDA 학습
    X_train_ss_pred = lda_ss.predict(X_train_ss)
    X_test_ss_pred = lda_ss.predict(X_test_ss)
    
    return accuracy_score(Y_train, X_train_ss_pred),accuracy_score(Y_test, X_test_ss_pred)

In [11]:
col_count = [25, 75, 125, 175, 225]

In [12]:
the_best_acc = 0
for i in col_count:
    print("num of columns :", i,'\n')
    for _ in range(50):
        sel_x = get_random_columns(x,i)
        acc1, acc2 = get_LDA(sel_x,y)
        if the_best_acc < acc2:
            the_best_acc = acc2
            best_x = sel_x
            print("renewal",acc1, acc2,"\n",sel_x.columns[:10])
    print()

num of columns : 25 

renewal 0.3442356342609324 0.33532068654019875 
 Index(['75', '217', '50', '161', '115', '154', '76', '197', '31', '231'], dtype='object')
renewal 0.38995301770870977 0.37651309846431796 
 Index(['178', '99', '237', '139', '0', '143', '64', '253', '103', '49'], dtype='object')
renewal 0.4041832309360318 0.3992773261065944 
 Index(['123', '27', '231', '95', '39', '72', '14', '166', '22', '138'], dtype='object')
renewal 0.4007047343693531 0.4041553748870822 
 Index(['80', '135', '40', '50', '11', '15', '44', '23', '105', '2'], dtype='object')
renewal 0.42397000361402243 0.41806684733514 
 Index(['154', '11', '134', '0', '218', '149', '32', '175', '61', '36'], dtype='object')

num of columns : 75 

renewal 0.49733465847488256 0.46648599819331527 
 Index(['148', '178', '250', '227', '110', '151', '157', '127', '17', '10'], dtype='object')
renewal 0.5401608239971087 0.5004516711833785 
 Index(['73', '80', '207', '189', '153', '158', '59', '246', '89', '177'], dtype='ob

In [13]:
from sklearn.preprocessing import StandardScaler
ss2 = StandardScaler()
best_x_ss = ss2.fit_transform(best_x)
test_ss = ss2.transform(test[best_x.columns])

In [14]:
lda = LinearDiscriminantAnalysis()
lda.fit(best_x_ss, y) # LDA 학습
pred = lda.predict(test_ss)

In [15]:
submit['label'] = pred
submit.to_csv("submit.csv", mode='w', header= True, index= False)